In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import scipy as sp
import scipy.signal
import radarkit
import matplotlib
import matplotlib.pyplot as plt
import time

In [ ]:
matplotlib.rcParams['font.family'] = 'serif'
matplotlib.rcParams['font.serif'] = ['Arial', 'Helvetica']
matplotlib.rcParams['font.sans-serif'] = ['System Font', 'Verdana', 'Arial']
matplotlib.rcParams['figure.figsize'] = (8.89, 5)   # Change the size of plots
matplotlib.rcParams['figure.dpi'] = 144

In [ ]:
sweep = radarkit.read('/Users/boonleng/Downloads/PX-20170220-050706-E2.4-Z.nc')

In [ ]:
z = sweep['moments'][0]['data']
p = sweep['moments'][4]['data']
r = sweep['moments'][5]['data']

In [ ]:
z.shape

In [ ]:
# Undo range correction and the ZCal in DSP
rr = 1.0e-3 * (np.arange(sweep['gateCount']) + 0.5) * sweep['gateSizeMeters']
snr = z - 20.0 * np.log10(rr) + 23

downSamplingRatio = int(sweep['gateSizeMeters'] / 30.0)

# Transition gate at 10.5 km at the given spacing
g = int((69.0e-6 * 3.0e8 * 0.5) / 30.0) + 5
k = np.zeros(g + downSamplingRatio, dtype=bool)
k[g+1:] = True
g = np.argmax(k[::downSamplingRatio])

print(g)

z_off = 10.0 * np.log10(1.5 / 67)
snr[:, :g] = snr[:, :g] + z_off

In [ ]:
# hl = plt.plot(snr.T[g-20:g+20, :])
hl = plt.plot(snr.T)

In [ ]:
plt.imshow(snr)
plt.clim(-10, 60)
plt.colorbar()

In [ ]:
# Map NAN to some finite numbers for SNR and PhiDP
snr = np.nan_to_num(snr, -100.0)
r = np.nan_to_num(r, 0.0)

### Fast Code

```python
s = time.time()
for _ in range(10):
    
    mask = np.logical_and(snr>0, r>0.85)

e = time.time()
print(e - s)
```

### Naive Code

```python
s = time.time()
for _ in range(10):
    
    mask = np.zeros(snr.shape, dtype=bool)
    for j in range(snr.shape[0]):
        for i in range(snr.shape[1]):
            if (snr[j, i] > 0 and r[j, i] > 0.85):
                mask[j, i] = True

e = time.time()
print(e - s)
```

In [ ]:
mask = np.logical_and(snr>0, r>0.85)
# p[:, :g] = p[:, :g] + 40;
# p[:, g:] = p[:, g:] + 44;

In [ ]:
plt.figure()

plt.subplot(121)
plt.imshow(p, cmap=matplotlib.cm.hsv)
plt.clim(-45, 45)
plt.title('PhiDP (Degrees)')
plt.colorbar(orientation='horizontal')

plt.subplot(122)
plt.imshow(mask, cmap=matplotlib.cm.gray)
plt.clim(0, 1)
plt.title('Mask: SNR > 0 AND RhoHV > 0.85')
plt.colorbar(orientation='horizontal')

In [ ]:
p

w = 10
r0 = np.zeros(p.shape[0])
for i in range(p.shape[0]):
    for g in range(p.shape[1] - w):
        kern = pm[i, g:g+w]
        if np.std(kern) < 10.0 and np.mean(np.diff(kern)) > 0.1:
            r0[i] = i
            break

pp = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
ww = np.ones(2) / 2.0
sp.signal.lfilter(ww, 1.0, pp)

In [ ]:
# Compute local slope
p_slope = sp.signal.lfilter(ww, 1.0, np.diff(p))
p_slope

In [ ]:
# Compute local variance
w = 10
ww = np.ones(w) / w;
ps = sp.signal.lfilter(ww, 1.0, pm)
p_var = sp.signal.lfilter(ww, 1.0, pm ** 2) - ps ** 2
p_var[~mask] = 100.0

# Compute local slope
p_slope = sp.signal.lfilter(ww, 1.0, np.diff(p))

In [ ]:
plt.figure()

yy = np.arange(p.shape[0])

plt.subplot(121)
plt.imshow(p_var, cmap=matplotlib.cm.tab20c)
plt.clim(0.0, 100.0)
plt.title('Local Variance')
plt.colorbar(orientation='horizontal')

plt.subplot(122)
plt.imshow(p_slope, cmap=matplotlib.cm.RdYlBu)
plt.clim(-1.0, 1.0)
plt.title('Local Slope')
plt.colorbar(orientation='horizontal')

In [ ]:
good = np.logical_and(p_var < 20.0, mask)
good[:, 1:] = np.logical_and(np.abs(np.nan_to_num(p_slope, 0.0)) < 0.5, good[:, 1:])
# good[:, 1:] = np.logical_and(np.nan_to_num(p_slope, 0.0) > 0.01, good[:, 1:])

good_count = np.sum(good, axis=1)

r0 = np.argmax(good[:, g+1:], axis=1) + g
rm = p.shape[1] - np.argmax(good[:, :g:-1], axis=1) - 1

paths = []
edge = np.zeros(good.shape, dtype=bool)
for i, s, e, c in zip(np.arange(p.shape[0]), r0, rm, good_count):
    if c > 50:
        paths.append((i, s, e))
        edge[i, s:e] = True;

In [ ]:
plt.figure()

plt.subplot(121)
plt.imshow(good, cmap=matplotlib.cm.gray)
plt.clim(0, 2)
plt.title('Good')
plt.colorbar(orientation='horizontal')

plt.subplot(122)
plt.imshow(edge, cmap=matplotlib.cm.gray)
# plt.plot(rm, yy, '.', markersize=1)
# plt.plot(r0, yy, '.', markersize=1)
plt.clim(0, 1)
plt.title('Good')
plt.colorbar(orientation='horizontal')

In [ ]:
rm.shape

In [ ]:
x = np.array([1, 2, 3, 4, 5])

In [ ]:
x[:0:-1]